## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-21-51-44 +0000,bbc,Three killed after bus crashes into Stockholm ...,https://www.bbc.com/news/articles/c629m7jm777o...
1,2025-11-14-21-48-00 +0000,wsj,Bitcoin Ends Week in a Slump,https://www.wsj.com/finance/currencies/bitcoin...
2,2025-11-14-21-47-49 +0000,nyt,Housing Official Who Attacked Democrats Invoke...,https://www.nytimes.com/2025/11/14/business/bi...
3,2025-11-14-21-47-00 +0000,wsj,The reopened federal government will on Thursd...,https://www.wsj.com/economy/jobs/september-job...
4,2025-11-14-21-46-32 +0000,nypost,NYC Mayor-elect Zohran Mamdani calls for Starb...,https://nypost.com/2025/11/14/business/nyc-may...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
67,trump,35
485,new,19
281,epstein,13
190,market,10
201,deal,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
42,2025-11-14-19-49-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...,101
23,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...,77
111,2025-11-14-16-56-00 +0000,wsj,New Prosecutor Named to Oversee Georgia Crimin...,https://www.wsj.com/us-news/law/new-prosecutor...,74
37,2025-11-14-20-17-56 +0000,bbc,US justice department investigates Epstein's a...,https://www.bbc.com/news/articles/cj97x2lere0o...,73
118,2025-11-14-16-31-31 +0000,nypost,Bondi directs SDNY Attorney Clayton to investi...,https://nypost.com/2025/11/14/us-news/trump-as...,70


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
42,101,2025-11-14-19-49-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...
57,49,2025-11-14-19-14-59 +0000,nypost,Man suspected of beating NYC super to death re...,https://nypost.com/2025/11/14/us-news/man-susp...
249,48,2025-11-14-04-38-11 +0000,nypost,JD Vance says surge of illegal immigrants drov...,https://nypost.com/2025/11/13/us-news/jd-vance...
77,41,2025-11-14-18-15-36 +0000,nypost,Olivia Nuzzi claims RFK Jr. told her ‘I love y...,https://nypost.com/2025/11/14/us-news/olivia-n...
3,35,2025-11-14-21-47-00 +0000,wsj,The reopened federal government will on Thursd...,https://www.wsj.com/economy/jobs/september-job...
276,34,2025-11-14-01-18-01 +0000,nypost,Canada man hijacks city bus but miraculously m...,https://nypost.com/2025/11/13/world-news/canad...
257,34,2025-11-14-03-37-55 +0000,nypost,NYC Mayor-elect Zohran Mamdani holds secretive...,https://nypost.com/2025/11/13/us-news/nyc-mayo...
26,32,2025-11-14-21-03-39 +0000,nypost,Purdue Pharma’s $7B opioid bankruptcy deal to ...,https://nypost.com/2025/11/14/business/judge-a...
228,32,2025-11-14-09-00-00 +0000,cbc,5-year-old Masa has been missing for more than...,https://www.cbc.ca/news/world/gaza-rubble-bodi...
12,31,2025-11-14-21-21-09 +0000,nyt,"Walmart CEO Doug McMillon to Retire, John Furn...",https://www.nytimes.com/2025/11/14/business/wa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
